In [ ]:
from math import *

In [ ]:
def update(mean1,var1,mean2,var2):
  new_mean = (var2 * mean1 + mean2 * var1)/(var1+var2)
  new_var =  1./(1./var1 + 1./var2)
  return [new_mean, new_var]

In [ ]:
print (update(10.,8.,13., 2.))

[12.4, 1.6]


In [ ]:
def predict(mean1,var1,mean2,var2):
  new_mean = mean1 + mean2
  new_var = var1 + var2
  return [new_mean, new_var]

In [ ]:
print(predict(10., 4., 12., 4.))

[22.0, 8.0]


In [ ]:
measurements = [5., 6., 7., 9., 10.]
motion = [1., 1., 2., 1., 1.]
measurement_sig = 4.
motion_sig = 2.
mu = 0.
sig = 10000.

In [ ]:
for n in range (len(measurements)):
  [mu,sig] = update(mu,sig,measurements[n],measurement_sig)
  print("Update :",[mu,sig])
  [mu,sig] = predict(mu,sig,motion[n],motion_sig)
  print("Predict :",[mu,sig])


Update : [4.998000799680128, 3.9984006397441023]
Predict : [5.998000799680128, 5.998400639744102]
Update : [5.999200191953932, 2.399744061425258]
Predict : [6.999200191953932, 4.399744061425258]
Update : [6.999619127420922, 2.0951800575117594]
Predict : [8.999619127420921, 4.09518005751176]
Update : [8.999811802788143, 2.0235152416216957]
Predict : [9.999811802788143, 4.023515241621696]
Update : [9.999906177177365, 2.0058615808441944]
Predict : [10.999906177177365, 4.005861580844194]


In [ ]:
class matrix:

    # implements basic operations of a matrix class

    def __init__(self, value):
        self.value = value
        self.dimx = len(value)
        self.dimy = len(value[0])
        if value == [[]]:
            self.dimx = 0

    def zero(self, dimx, dimy):
        # check if valid dimensions
        if dimx < 1 or dimy < 1:
            raise (ValueError, "Invalid size of matrix")
        else:
            self.dimx = dimx
            self.dimy = dimy
            self.value = [[0 for row in range(dimy)] for col in range(dimx)]

    def identity(self, dim):
        # check if valid dimension
        if dim < 1:
            raise (ValueError, "Invalid size of matrix")
        else:
            self.dimx = dim
            self.dimy = dim
            self.value = [[0 for row in range(dim)] for col in range(dim)]
            for i in range(dim):
                self.value[i][i] = 1

    def show(self):
        for i in range(self.dimx):
            print(self.value[i])
        print(' ')

    def __add__(self, other):
        # check if correct dimensions
        if self.dimx != other.dimx or self.dimy != other.dimy:
            raise (ValueError, "Matrices must be of equal dimensions to add")
        else:
            # add if correct dimensions
            res = matrix([[]])
            res.zero(self.dimx, self.dimy)
            for i in range(self.dimx):
                for j in range(self.dimy):
                    res.value[i][j] = self.value[i][j] + other.value[i][j]
            return res

    def __sub__(self, other):
        # check if correct dimensions
        if self.dimx != other.dimx or self.dimy != other.dimy:
            raise (ValueError, "Matrices must be of equal dimensions to subtract")
        else:
            # subtract if correct dimensions
            res = matrix([[]])
            res.zero(self.dimx, self.dimy)
            for i in range(self.dimx):
                for j in range(self.dimy):
                    res.value[i][j] = self.value[i][j] - other.value[i][j]
            return res

    def __mul__(self, other):
        # check if correct dimensions
        if self.dimy != other.dimx:
            raise (ValueError, "Matrices must be m*n and n*p to multiply")
        else:
            # multiply if correct dimensions
            res = matrix([[]])
            res.zero(self.dimx, other.dimy)
            for i in range(self.dimx):
                for j in range(other.dimy):
                    for k in range(self.dimy):
                        res.value[i][j] += self.value[i][k] * other.value[k][j]
            return res

    def transpose(self):
        # compute transpose
        res = matrix([[]])
        res.zero(self.dimy, self.dimx)
        for i in range(self.dimx):
            for j in range(self.dimy):
                res.value[j][i] = self.value[i][j]
        return res

    # Thanks to Ernesto P. Adorio for use of Cholesky and CholeskyInverse functions

    def Cholesky(self, ztol=1.0e-5):
        # Computes the upper triangular Cholesky factorization of
        # a positive definite matrix.
        res = matrix([[]])
        res.zero(self.dimx, self.dimx)

        for i in range(self.dimx):
            S = sum([(res.value[k][i])**2 for k in range(i)])
            d = self.value[i][i] - S
            if abs(d) < ztol:
                res.value[i][i] = 0.0
            else:
                if d < 0.0:
                    raise (ValueError, "Matrix not positive-definite")
                res.value[i][i] = sqrt(d)
            for j in range(i+1, self.dimx):
                S = sum([res.value[k][i] * res.value[k][j] for k in range(self.dimx)])
                if abs(S) < ztol:
                    S = 0.0
                try:
                   res.value[i][j] = (self.value[i][j] - S)/res.value[i][i]
                except:
                   raise (ValueError, "Zero diagonal")
        return res

    def CholeskyInverse(self):
        # Computes inverse of matrix given its Cholesky upper Triangular
        # decomposition of matrix.
        res = matrix([[]])
        res.zero(self.dimx, self.dimx)

        # Backward step for inverse.
        for j in reversed(range(self.dimx)):
            tjj = self.value[j][j]
            S = sum([self.value[j][k]*res.value[j][k] for k in range(j+1, self.dimx)])
            res.value[j][j] = 1.0/tjj**2 - S/tjj
            for i in reversed(range(j)):
                res.value[j][i] = res.value[i][j] = -sum([self.value[i][k]*res.value[k][j] for k in range(i+1, self.dimx)])/self.value[i][i]
        return res

    def inverse(self):
        aux = self.Cholesky()
        res = aux.CholeskyInverse()
        return res

    def __repr__(self):
        return repr(self.value)



In [ ]:
############################################
### use the code below to test your filter!
############################################

measurements = [1, 2, 3]

x = matrix([[0.], [0.]]) # initial state (location and velocity)
P = matrix([[1000., 0.], [0., 1000.]]) # initial uncertainty
u = matrix([[0.], [0.]]) # external motion
F = matrix([[1., 1.], [0, 1.]]) # next state function
H = matrix([[1., 0.]]) # measurement function
R = matrix([[1.]]) # measurement uncertainty
I = matrix([[1., 0.], [0., 1.]]) # identity matrix


In [ ]:
########################################

# Implement the filter function below

def kalman_filter(x, P):
    for n in range(len(measurements)):


        # measurement update
        Z = matrix([[measurements[n]]])
        y= Z- (H*x)
        S = H*P*H.transpose() + R
        K = P*H.transpose() * S.inverse()
        x= x + (K*y)
        P = (I-(K*H))*P
        #print(x)
        #print(P)


        # prediction
        x= (F*x) + u
        P = F * P * F.transpose()
        #print(x)
        #print(P)

    return x,P



In [ ]:

print(kalman_filter(x, P))
# output should be:
# x: [[3.9996664447958645], [0.9999998335552873]]
# P: [[2.3318904241194827, 0.9991676099921091], [0.9991676099921067, 0.49950058263974184]]

([[3.9996664447958645], [0.9999998335552873]], [[2.3318904241194827, 0.9991676099921091], [0.9991676099921067, 0.49950058263974184]])


A Note About the State Transition Function: Bu
If you go back to the video, you'll notice that the state transition function was first given as
$x' = Fx + Bu + \nu. $

But then Bu was crossed out leaving $x' = Fx + \nu$.

B is a matrix called the control input matrix and u is the control vector.

As an example, let's say we were tracking a car and we knew for certain how much the car's motor was going to accelerate or decelerate over time; in other words, we had an equation to model the exact amount of acceleration at any given moment. Bu would represent the updated position of the car due to the internal force of the motor. We would use \nu to represent any random noise that we could not precisely predict like if the car slipped on the road or a strong wind moved the car.

For the Kalman filter lessons, we will assume that there is no way to measure or know the exact acceleration of a tracked object. For example, if we were in an autonomous vehicle tracking a bicycle, pedestrian or another car, we would not be able to model the internal forces of the other object; hence, we do not know for certain what the other object's acceleration is. Instead, we will set Bu = 0 and represent acceleration as a random noise with mean $\nu. $

The State Transition Matrix
$ \begin{pmatrix} p_x' \\\ p_y' \\\ v_x' \\\ v_y' \\\ \end{pmatrix} = \begin{pmatrix} 1 & 0 & \Delta t & 0 \\\ 0 & 1 & 0 & \Delta t \\\ 0 & 0 & 1 & 0 \\\ 0 & 0 & 0 & 1 \end{pmatrix} \begin{pmatrix} p_x \\\ p_y \\\ v_x \\\ v_y \end{pmatrix} + \begin{pmatrix} \nu_{px} \\\\ \nu_{py} \\\\ \nu_{vx} \\\\ \nu_{vy} \end{pmatrix}$
As a reminder, the above equation is $x' = Fx + noise $

Process Covariance Matrix Q - Intuition
As a reminder, here are the state covariance matrix update equation and the equation for Q.

P' = FPF^T + Q

$ Q = \begin{pmatrix} \frac{\Delta t^4}{{4}}\sigma_{ax}^2 & 0 & \frac{\Delta t^3}{{2}}\sigma_{ax}^2 & 0 \\ 0 & \frac{\Delta t^4}{{4}}\sigma_{ay}^2 & 0 & \frac{\Delta t^3}{{2}}\sigma_{ay}^2 \\ \frac{\Delta t^3}{{2}}\sigma_{ax}^2 & 0 & \Delta t^2\sigma_{ax}^2 & 0 \\ 0 & \frac{\Delta t^3}{{2}}\sigma_{ay}^2 & 0 & \Delta t^2\sigma_{ay}^2 \end{pmatrix} $

Because our state vector only tracks position and velocity, we are modeling acceleration as a random noise. The Q matrix includes time \Delta t to account for the fact that as more time passes, we become more uncertain about our position and velocity. So as \Delta t increases, we add more uncertainty to the state covariance matrix P.

Combining both 2D position and 2D velocity equations previously deducted formulas we have:

$\begin{cases} p_x' = p_x + v_x \Delta t + \frac{a_x \Delta t^2}{{2}}\\ p_y' = p_y + v_y \Delta t + \frac{a_y \Delta t^2}{{2}}\\ v_x' = v_x + a_x \Delta t\\ v_y' = v_y + a_y \Delta t \end{cases} $

Since the acceleration is unknown we can add it to the noise component, and this random noise would be expressed analytically as the last terms in the equation derived above. So, we have a random acceleration vector \nu in this form:

$\nu = \begin{pmatrix} \nu_{px} \\ \nu_{py} \\ \nu_{vx} \\ \nu_{vy} \end{pmatrix} = \begin{pmatrix} \frac{a_x \Delta t^2}{{2}} \\ \frac{a_y \Delta t^2}{{2}} \\ a_x \Delta t \\ a_y \Delta t \end{pmatrix}$
which is described by a zero mean and a covariance matrix Q, so $\nu \sim N(0,Q)$.

The vector \nu can be decomposed into two components a 4 by 2 matrix G which does not contain random variables and a 2 by 1 matrix a which contains the random acceleration components:

$\nu = \begin{pmatrix} \frac{a_x \Delta t^2}{{2}} \\ \frac{a_y \Delta t^2}{{2}} \\ a_x \Delta t \\ a_y \Delta t \end{pmatrix} = \begin{pmatrix} \frac{\Delta t^2}{{2}} & 0 \\ 0 & \frac{\Delta t^2}{{2}} \\ \Delta t & 0 \\ 0 & \Delta t \end{pmatrix} \begin{pmatrix} a_x\\ a_y \end{pmatrix} = Ga
\Delta t $
 is computed at each Kalman Filter step and the acceleration is a random vector with zero mean and standard deviations $ \sigma_{ax_{}}$ and $\sigma_{ay}$.

Based on our noise vector we can define now the new covariance matrix Q.
The covariance matrix is defined as the expectation value of the noise vector \nu times the noise vector $\nu^T$. So let’s write this down:

$ Q = E[\nu \nu^T] = E[Gaa^TG^T]$

As G does not contain random variables, we can put it outside the expectation calculation.

$ Q = G E[aa^T] G^T = G \begin{pmatrix} \sigma_{ax}^2 & \sigma_{axy} \\ \sigma_{axy} & \sigma_{ay}^2 \end{pmatrix} G^T = G Q_{\nu} G^T $


This leaves us with three statistical moments:

the expectation of ax times ax, which is the variance of ax squared: $ \sigma_{ax}^2$ .
the expectation of ay times ay, which is the variance of ay squared: $ \sigma_{ay}^2$.
and the expectation of ax times ay, which is the covariance of ax and ay: $ \sigma_{axy}$.
$ a_{x_{}}$ and $a_{y_{}} $ are assumed uncorrelated noise processes. This means that the covariance $ \sigma_{axy_{}}$ in $ Q_{\nu} $ is zero:

$ Q_{\nu} = \begin{pmatrix} \sigma_{ax}^2 & \sigma_{axy} \\ \sigma_{axy} & \sigma_{ay}^2 \end{pmatrix} = \begin{pmatrix} \sigma_{ax}^2 & 0 \\ 0 & \sigma_{ay}^2 \end{pmatrix} $


So after combining everything in one matrix we obtain our 4 by 4 Q matrix:

$ Q = G Q_{\nu} G^T = \begin{pmatrix} \frac{\Delta t^4}{{4}}\sigma_{ax}^2 & 0 & \frac{\Delta t^3}{{2}}\sigma_{ax}^2 & 0 \\ 0 & \frac{\Delta t^4}{{4}}\sigma_{ay}^2 & 0 & \frac{\Delta t^3}{{2}}\sigma_{ay}^2 \\ \frac{\Delta t^3}{{2}}\sigma_{ax}^2 & 0 & \Delta t^2\sigma_{ax}^2 & 0 \\ 0 & \frac{\Delta t^3}{{2}}\sigma_{ay}^2 & 0 & \Delta t^2\sigma_{ay}^2 \end{pmatrix} $

Deriving the Radar Measurement Function
The measurement function is composed of three components that show how the predicted state, $ x' = (p_x', p_y', v_x', v_y')^T$, is mapped into the measurement space, $ z = (\rho, \varphi, \dot{\rho})^T:$

The range, $ \rho $, is the distance to the pedestrian which can be defined as:

$ \rho = \sqrt[]{p_x^2 + p_y^2}$

$ \varphi $ is the angle between \rho and the x direction and can be defined as:

$ \varphi = \arctan(p_y/p_x) $
There are two ways to do the range rate $ \dot{\rho(t)} $ derivation:

Generally we can explicitly describe the range, \rho, as a function of time:

$ \rho(t) = \sqrt{p_x(t)^2 + p_y(t)^2} $
The range rate, $ \dot{\rho(t)} $, is defined as time rate of change of the range, $ \rho $, and it can be described as the time derivative of $ \rho $:

$ \dot{\rho} = \frac{\partial \rho(t)}{\partial t} = \frac{ \partial}{\partial t}\sqrt{p_x(t)^2 + p_y(t)^2} = \frac{1}{2 \sqrt{p_x(t)^2 + p_y(t)^2}} (\frac{ \partial}{\partial t}p_x(t)^2 + \frac{ \partial}{\partial t}p_y(t)^2) $

$ =\frac{1}{2 \sqrt{p_x(t)^2 + p_y(t)^2}} (2 p_x(t) \frac{ \partial}{\partial t} p_x(t) + 2 p_y(t) \frac{ \partial}{\partial t} p_y(t)) $

$ \frac{ \partial}{\partial t} p_x(t) $ is nothing else than $ v_x(t)$, similarly $ \frac{ \partial}{\partial t} p_y(t) $ is $ v_y(t) $. So we have:

$ \dot{\rho} = \frac{\partial \rho(t)}{\partial t} = \frac{1}{2 \sqrt{p_x(t)^2 + p_y(t)^2}} (2 p_x(t) v_x(t) + 2 p_y(t) v_y(t)) = \frac{2( p_x(t) v_x(t) + p_y(t) v_y(t))}{2 \sqrt{p_x(t)^2 + p_y(t)^2}} $

$ =\frac{p_x(t) v_x(t) + p_y(t) v_y(t)}{ \sqrt{p_x(t)^2 + p_y(t)^2}} $

For simplicity we just use the following notation:

$ \dot{\rho} = \frac{p_x v_x + p_y v_y}{ \sqrt{p_x^2 + p_y^2}} $
The range rate,  $ \dot{\rho} $, can be seen as a scalar projection of the velocity vector, $ \vec{v} $, onto $ \vec{\rho} $. Both $ \vec{\rho}$  and $ \vec{v}$ are 2D vectors defined as:

$ \vec{\rho}= \begin{pmatrix} p_x\\ p_y \end{pmatrix}, \space \vec{v}=\begin{pmatrix} v_x\\ v_y \end{pmatrix}$
The scalar projection of the velocity vector $ \vec{v}$ onto $ \vec{\rho}$ is defined as:

$ \dot{\rho}= \frac{\vec{v} \vec{\rho}}{\lvert \vec{\rho} \rvert} = \frac{ \begin{pmatrix} v_x & v_y \end{pmatrix} \begin{pmatrix} p_x\\ p_y \end{pmatrix} }{ \sqrt{p_x^2 + p_y^2} } = \frac{p_x v_x + p_y v_y}{ \sqrt{p_x^2 + p_y^2}} $
where $ \lvert \vec{\rho} \rvert $ is the length of $ \vec{\rho}$. In our case it is actually the range, so $ \rho = \lvert \vec{\rho} \rvert$ .

cheat sheet



$ h(x) \approx h(\mu) + \frac{\partial h(\mu)}{\partial x}(x-\mu) = \arctan(\mu) + \frac{1}{1+\mu^2}(x-\mu) $


In our example $\mu=0$, therefore:

$h(x) \approx \arctan(0) + \frac{1}{1+0}(x-0) = x$
So, the function, $ h(x) = \arctan(x)$, will be approximated by a line:
$ h(x) \approx x.$

And now, let's keep going!

Multivariate Taylor Series
Now that you’ve seen how to do a Taylor series expansion with a one-dimensional equation, we’ll need to look at the Taylor series expansion for multi-dimensional equations. Recall from the Radar Measurements lecture that the h function is composed of three equations that show how the predicted state, $ x' = (p_x', p_y', v_x', v_y')^T $, is mapped into the measurement space, $ z = (\rho, \varphi, \dot{\rho})^T$ :

$ h(x')= \begin{pmatrix} \sqrt{ p{'}_x^2 + p{'}_y^2 }\\ \arctan(p_y' / p_x')\\ \frac{p_x' v_x' + p_y' v_y'}{\sqrt{p{'}_x^2 + p{'}_{y}^2}} \end{pmatrix} $




These are multi-dimensional equations, so we will need to use a multi-dimensional Taylor series expansion to make a linear approximation of the h function. Here is a general formula for the multi-dimensional Taylor series expansion:
$
T(x) = f(a) + (x - a)^TDf(a) + \frac 1{2!}(x-a)^T{D^2f(a)}(x - a) + ... $
where Df(a) is called the Jacobian matrix and D^2f(a) is called the Hessian matrix.

They represent first order and second order derivatives of multi-dimensional equations. A full Taylor series expansion would include higher order terms as well for the third order derivatives, fourth order derivatives, and so on.

Notice the similarities between the multi-dimensional Taylor series expansion and the one-dimensional Taylor series expansion:

$T(x) = f(a) + \frac{f'(a)}{1!}(x - a) + \frac{f''(a)}{2!}(x-a)^2 + \frac{f'''(a)}{3!}(x - a)^3 + ... $
To derive a linear approximation for the h function, we will only keep the expansion up to the Jacobian matrix Df(a). We will ignore the Hessian matrix D^2f(a) and other higher order terms. Assuming (x - a) is small, (x - a)^2 or the multi-dimensional equivalent (x-a)^T (x - a) will be even smaller; the extended Kalman filter we'll be using assumes that higher order terms beyond the Jacobian are negligible.

Let's first calculate the Jacobian matrix for the extended Kalman filter. Then we'll show the difference between the Kalman filter equations and the extended Kalman filter equations.$

We're going to calculate, step by step, all the partial derivatives in H_j:

$\Large H_j = \begin{bmatrix} \frac{\partial \rho}{\partial p_x} & \frac{\partial \rho}{\partial p_y} & \frac{\partial \rho}{\partial v_x} & \frac{\partial \rho}{\partial v_y}\\ \frac{\partial \varphi}{\partial p_x} & \frac{\partial \varphi}{\partial p_y} & \frac{\partial \varphi}{\partial v_x} & \frac{\partial \varphi}{\partial v_y}\\ \frac{\partial \dot{\rho}}{\partial p_x} & \frac{\partial \dot{\rho}}{\partial p_y} & \frac{\partial \dot{\rho}}{\partial v_x} & \frac{\partial \dot{\rho}}{\partial v_y} \end{bmatrix}$
So all of H_j's elements are calculated as follows:
$
\frac{\partial \rho}{\partial p_x}= \frac{\partial}{\partial p_x}(\sqrt[]{p_x^2 + p_y^2}) = \frac{2 p_x}{2 \sqrt[]{p_x^2 + p_y^2}} = \frac{p_x}{\sqrt[]{p_x^2 + p_y^2}}$

$
\frac{\partial \rho}{\partial p_y}= \frac{\partial}{\partial p_y}(\sqrt[]{p_x^2 + p_y^2}) = \frac{2 p_y}{2 \sqrt[]{p_x^2 + p_y^2}} = \frac{p_y}{\sqrt[]{p_x^2 + p_y^2}}$


$
\frac{\partial \rho}{\partial v_x} = \frac{\partial}{\partial v_x}(\sqrt[]{p_x^2 + p_y^2}) = 0$



$\frac{\partial \rho}{\partial v_y} = \frac{\partial}{\partial v_y}(\sqrt[]{p_x^2 + p_y^2}) = 0$


$
\frac{\partial \varphi}{\partial p_x} = \frac{\partial}{\partial p_x}arctan(p_y/p_x) = \frac{1}{(\frac{p_y}{p_x})^2 + 1} (- \frac{p_y}{p_x^2}) = -\frac{p_y}{p_x^2+p_y^2}$



$\frac{\partial \varphi}{\partial p_y} = \frac{\partial}{\partial p_y}arctan(p_y/p_x) = \frac{1}{(\frac{p_y}{p_x})^2 + 1} (\frac{1}{p_x}) = \frac{p_x^2}{p_x^2+p_y^2} \frac{1}{p_x} =\frac{p_x}{p_x^2+p_y^2}$



$\frac{\partial \varphi}{\partial v_x} = \frac{\partial}{\partial v_x}arctan(p_y/p_x) = 0$


$\frac{\partial \varphi}{\partial v_y} = \frac{\partial}{\partial v_y}arctan(p_y/p_x) = 0$



$\frac{\partial \dot{\rho}}{\partial p_x} = \frac{\partial}{\partial p_x} \left( \frac{p_x v_x + p_y v_y}{\sqrt[]{p_x^2 + p_y^2}} \right) $


In order to calculate the derivative of this function we use the quotient rule.

Given a function z that is quotient of two other functions, f and g:

$ z = \frac{f}{g} $
its derivative with respect to x is defined as:

$\frac{\partial z}{\partial x} = \frac { \frac{\partial f}{\partial x} g - \frac{\partial g}{\partial x} f } {g^2}$
in our case:

$f = p_x v_x + p_y v_y
g = \sqrt[]{p_x^2 + p_y^2}$


Their derivatives are:

$\frac{\partial f}{\partial p_x} = \frac{\partial}{\partial p_x} (p_x v_x + p_y v_y) = v_x
\frac{\partial g}{\partial p_x} = \frac{\partial}{\partial p_x} \left( \sqrt[]{p_x^2 + p_y^2} \right) = \frac{p_x}{\sqrt[]{p_x^2 + p_y^2}}$


Putting everything together into the derivative quotient rule we have:

 $\frac{\partial \dot{\rho}} {\partial p_x} = \frac{v_x \sqrt[]{p_x^2 + p_y^2} - \frac{p_x}{\sqrt[]{p_x^2 + p_y^2}} (p_x v_x + p_y v_y)} {p_x^2 + p_y^2} = \frac{p_y(v_x p_y - v_y p_x)}{(p_x^2 + p_y^2)^{3/2}}$


Similarly,

$\frac{\partial \dot{\rho}}{\partial p_y} = \frac{\partial}{\partial p_y} \left( \frac{p_x v_x + p_y v_y}{\sqrt[]{p_x^2 + p_y^2}} \right) = \frac{p_x(v_y p_x - v_x p_y)}{(p_x^2 + p_y^2)^{3/2}}
\frac{\partial \dot{\rho}}{\partial v_x} = \frac{\partial}{\partial v_x} \left( \frac{p_x v_x + p_y v_y}{\sqrt[]{p_x^2 + p_y^2}} \right) = \frac{p_x}{\sqrt[]{p_x^2 + p_y^2}}
\frac{\partial \dot{\rho}}{\partial v_y} = \frac{\partial}{\partial v_y} \left( \frac{p_x v_x + p_y v_y}{\sqrt[]{p_x^2 + p_y^2}} \right) = \frac{p_y}{\sqrt[]{p_x^2 + p_y^2}}$


So now, after calculating all the partial derivatives, our resulted Jacobian, H_j is:

$\Large H_j = \begin{bmatrix} \frac{p_x}{\sqrt[]{p_x^2 + p_y^2}} & \frac{p_y}{\sqrt[]{p_x^2 + p_y^2}} & 0 & 0\\ -\frac{p_y}{p_x^2 + p_y^2} & \frac{p_x}{p_x^2 + p_y^2} & 0 & 0\\ \frac{p_y(v_x p_y - v_y p_x)}{(p_x^2 + p_y^2)^{3/2}} & \frac{p_x(v_y p_x - v_x p_y)}{(p_x^2 + p_y^2)^{3/2}} & \frac{p_x}{\sqrt[]{p_x^2 + p_y^2}} & \frac{p_y}{\sqrt[]{p_x^2 + p_y^2}}\\ \end{bmatrix}$

Extended Kalman Filter Equations
Although the mathematical proof is somewhat complex, it turns out that the Kalman filter equations and extended Kalman filter equations are very similar. The main differences are:

the F matrix will be replaced by $ F_j $when calculating P'.
the H matrix in the Kalman filter will be replaced by the Jacobian matrix $ H_j  $ when calculating S, K, and P.
to calculate $ x' $, the prediction update function, f, is used instead of the F matrix.
to calculate y, the h function is used instead of the H matrix.
For this project, however, we do not need to use the f function or F_j. If we had been using a non-linear model in the prediction step, we would need to replace the F matrix with its Jacobian, F_j. However, we are using a linear model for the prediction step. So, for the prediction step, we can still use the regular Kalman filter equations and the F matrix rather than the extended Kalman filter equations.

The measurement update for lidar will also use the regular Kalman filter equations, since lidar uses linear equations. Only the measurement update for the radar sensor will use the extended Kalman filter equations.

One important point to reiterate is that the equation $ y = z - Hx' $ for the Kalman filter does not become $ y = z - H_jx $ for the extended Kalman filter. Instead, for extended Kalman filters, we'll use the h function directly to map predicted locations x' from Cartesian to polar coordinates.

The comparison for reference.
The comparison for reference.

Clarification of $ u = 0 $
In the above image, the prediction equation is written as $ x' = Fx + u $ and $ x' = f(x,u) $ . Previously the equation was written $ x' = Fx + \nu$.

It is just a question of notation where $ \nu $ is the greek letter $ "nu" $ and "u" is used in the code examples. Remember that $ \nu $ is represented by a gaussian distribution with mean zero. The equation $ x' = Fx + u $ or the equivalent equation $ x' = Fx + \nu $ calculates the mean value of the state variable x; hence we set u = 0. The uncertainty in the gaussian distribution shows up in the Q matrix.

More Details About Calculations with Radar Versus Lidar
In the radar update step, the Jacobian matrix H_j is used to calculate S, K and P. To calculate y, we use the equations that map the predicted location x' from Cartesian coordinates to polar coordinates:

$ h(x')= \begin{pmatrix} \sqrt{ p{'}_{x}^2 + p{'}_{y}^2 }\\ \arctan(p_y' / p_x')\\ \frac{p_x' v_x' + p_y' v_y'}{\sqrt{p{'}_{x}^2 + p{'}_{y}^2}} \end{pmatrix} $
The predicted measurement vector x' is a vector containing values in the form $ \begin{bmatrix} p_x, p_y, v_x, v_y \end{bmatrix} $. The radar sensor will output values in polar coordinates:

$ \begin{pmatrix} \rho\\ \phi\\ \dot{\rho} \end{pmatrix} $
In order to calculate y for the radar sensor, we need to convert x' to polar coordinates. In other words, the function h(x) maps values from Cartesian coordinates to polar coordinates. So the equation for radar becomes $ y = z_{radar} - h(x')$.

One other important point when calculating y with radar sensor data: the second value in the polar coordinate vector is the angle $ \phi $. You'll need to make sure to normalize $ \phi$ in the y vector so that its angle is between $ -\pi $ and $ \pi $; in other words, add or subtract $ 2\pi $ from $ \phi$ until it is between $ -\pi $ and $ \pi $.


1) What is the influence of $ \nu_{a,k_{}}$ and $\nu_{\ddot{\psi},k} $ on the velocity?
In other words, what is c in the process model?

A.$ \Large \Delta t \cdot \nu_{\ddot{\psi},k}$

B. $ \Large \Delta t \cdot \nu_{a,k} $

1) How will the noise processes influence the velocity? Select one of the options from above (👆).


A


B

SOLUTION:
2) What is the influence of $ \nu_{a,k_{}}$ and $ \nu_{\ddot{\psi},k} $ on the yaw angle?
In other words, what is d in the process model?

A. $ \Large \frac{1}{2}(\Delta t)^2 \cdot \nu_{\ddot{\psi},k}$

B. $ \Large \Delta t \cdot \nu_{\ddot{\psi},k}$


2) How will the noise processes influence the yaw angle? Select one of the options from above (👆).


A


B

SOLUTION:
3) What is the influence of $ \nu_{a,k_{}}$ and $ \nu_{\ddot{\psi},k}$ on the yaw rate?
In other words, what is \large e in the process model?

A. $ \Large \Delta t \cdot \nu_{a,k}$

B. $ \Large \Delta t \cdot \nu_{\ddot{\psi},k}$


3) How will the noise processes influence the yaw rate? Select one of the options from above (👆).


1) What would the x acceleration offset be if the car were driving perfectly straight? In other words, what is a?


A. $ \frac{1}{2}(\Delta t)^2 cos(\psi_k) \cdot \nu_{a,k}$

B. $\frac{1}{2}(\Delta t)^2 sin(\psi_k) \cdot \nu_{a,k}$


1) Which of the above (👆) options represents the acceleration x offset of a car driving straight at time k+1?


A


B

SOLUTION:
1) What would the y acceleration offset be if the car were driving perfectly straight? In other words, what is b?


A. $ \frac{1}{2}(\Delta t)^2 cos(\psi_k) \cdot \nu_{a,k} $

B. $ \frac{1}{2}(\Delta t)^2 sin(\psi_k) \cdot \nu_{a,k} $


2) Which of the above (👆) options represents the acceleration y offset of a car driving straight at time k+1?


A


B

State Vector
$ x_{k+1|k}=\begin{bmatrix} p_x\\ p_y\\ v\\ \psi\\ \dot{\psi} \end{bmatrix} $
Measurement Vector
$ z_{k+1|k}=\begin{bmatrix} \rho\\ \varphi\\ \dot{\rho} \end{bmatrix}$
Measurement Model
$ z_{k+1|k}=h(x_{k+1}) + w_{k+1}$
$ \rho = \sqrt{p_x^2+p_y^2}$
$ \varphi =arctan(\frac{p_y}{p_x})$
$ \dot{\rho}=\frac{p_xcos(\psi)v+p_ysin(\psi)v}{\sqrt{p_x^2+p_y^2}}$
Predicted Measurement Mean
$ \large z_{k+1|k} = \sum_{i=1}^{n_\sigma} w_i Z_{k+1|k,i}$
Predicted Covariance
$ \large S_{k+1|k} = \sum_{i=1}^{n_\sigma} w_i( Z_{k+1|k,i} - z_{k+1|k})(Z_{k+1|k,i} - z_{k+1|k})^T + R$
$ \large R = E(w_k\cdot w_k^T) = \begin{bmatrix} \sigma_{\rho}^2 \qquad 0\qquad0\\ 0\qquad\sigma_{\varphi}^2 \qquad 0\\ 0\qquad0\qquad\sigma_{\dot{\rho}}^2 \end{bmatrix}$